In [60]:
import pandas as pd
import IPython.display as ipd
import numpy as np

#splitting the data into multiple data sets for ease of different visualisations and focuses
raw_data = pd.read_csv('patient_risk_profiles.csv')
sex = ['Male','Female']
raw_data = raw_data.set_index('personId')
age_groups = ['10-14','15-19','20-24','65-69','40-44','45-49','55-59','85-89','75-79','5-9','25-29','0-4','70-74','50-54','60-64','35-39','30-34','80-84','90-94']

#this contains just the person ID and their ages
age_data = raw_data.iloc[:,:19]
age_list_order = list(age_data.columns)

#this contains just the person ID and their sexes
sex_data = raw_data.iloc[:,[19,20]]

#This contains the person ID and all the predicted risk data for the 14 different conditions
prediction_data = raw_data.iloc[:,-14:]

#this contains the person ID and all the data on the 'in the prior year' factors
raw_data.shape
factors_data = raw_data.iloc[:,21:85]
factors_data.to_csv('factors_data.csv')


#First replacing the binary 1 (i.e. yes) with the age group that person is in (i.e. the column header)
for i in age_data.columns:
  age_data[i].replace(1, i, inplace=True)

#Renaming all the age groups to just the numbers (e.g. '5-9' instead of 'age group: 5 -  9')
for i in range(len(age_list_order)):
  age_data.replace(age_list_order[i], age_groups[i], inplace=True)

#person_ages still has a column for each age group at this point, but any 0 reads as NaN and any 1 reads as the relevant group
person_ages = age_data.replace(0, np.nan).reset_index()
person_ages = person_ages.melt(id_vars='personId',value_vars=age_data.columns[1:],var_name='columns',value_name='Age group (years)')
person_ages = person_ages.set_index('personId') #rearranging and indexing to allow for any NaN to be dropped
person_ages = person_ages['Age group (years)'].dropna().to_frame()
person_ages = person_ages.sort_index() #sorting again from perosn 1 to person 100
ipd.display(person_ages) #only 96 rows - some people don't have age data
#person_ages now contains only 2 columns: one for person ID and one for the age group


#this is to, simliar to the age data, put the sex directly next to the relevant person rather than spread over 2 columns
for i in sex_data.columns:
  sex_data[i].replace(1, i, inplace=True)

sex = ['Female','Male']
#reassigning 'sex: female' and sex: 'male' to just 'female' and 'male'
for i in range(len(sex)):
  sex_data.replace(sex_data.columns[i], sex[i], inplace=True)

person_sexes = sex_data.replace(0, np.nan).reset_index()
person_sexes = person_sexes.melt(id_vars='personId',value_vars=sex_data.columns,var_name='columns',value_name='Sex')
person_sexes = person_sexes.set_index('personId')
person_sexes = person_sexes['Sex'].dropna().to_frame()
person_sexes = person_sexes.sort_index()#109 rows at this stage - at least 9 people have been marked as both sex (more if some others are missing data)
person_sexes = person_sexes.groupby('personId').agg(func=list) #aggregating to reduce to 100 lines - all persons have data on sex, any with both 'male' and 'female' assignments have it listed
person_sexes.to_csv('person_sexes.csv')
#person_sexes now contains only 2 columns: one for person ID and one for the sex

person_sexes = pd.read_csv('person_sexes.csv') #saved and re-read in order for .replace() to recognise the lists as singular values
person_sexes = person_sexes.replace(["['Male', 'Female']" ,"['Female', 'Male']"], 'Other')
person_sexes = person_sexes.replace("['Male']", 'Male')
person_sexes = person_sexes.replace("['Female']", 'Female')



factors_data_tf = factors_data.copy()
#to keep one set with the 1/0 as it might be easier for some manipulation later on, and some with either True'False or the list of factors

for i in factors_data_tf.columns:
  factors_data_tf[i].replace(1, i, inplace=True)

person_factors = factors_data_tf.replace(0, np.nan).reset_index()
person_factors = person_factors.melt(id_vars='personId',value_vars=factors_data.columns,var_name='columns',value_name='Factors')
person_factors = person_factors.set_index('personId')
person_factors = person_factors['Factors'].dropna().to_frame()
person_factors = person_factors.sort_index()
ipd.display(person_factors)
#person_factors contains 2 columns: one for person ID and one for each factor the person has experienced (one on each row, so potentially multiple rows for each person)


for i in factors_data_tf.columns:
  factors_data_tf.replace(i, 'True', inplace=True)
  factors_data_tf.replace(0, 'False', inplace=True)
    #this is for the True/False format


grouped_factors = person_factors.groupby('personId').agg(func=list)
#this means each person now has 1 row (still onyl 2 columns) so the factor column contains a list
ipd.display(grouped_factors) #only 96 rows - not everyone has (at least) one of the factors

print(list(grouped_factors.index.values)) #shows a list of all those who have experienced one of the factors


identifiers = person_ages.merge(person_sexes, on='personId', how='right') #how='right' to keep data for all examples, even those missing age data
#identifiers contains 3 columns: person ID, age group, and sex

id_factors = identifiers.merge(grouped_factors, on='personId', how='left')
#id_factors contains 4 columns: person ID, age group, sex, and the list of factors experienced by that person

short_data = id_factors.merge(prediction_data, on='personId')
ipd.display(short_data)
#short_data contains 18 columns: person ID, age group, sex, the list of factors, and one column for each of the 14 predicted risks
short_data.to_csv('short_data.csv') #saving this locally such that it can be used for the visualisation files

factors_preds = factors_data.merge(prediction_data, on='personId')
ipd.display(factors_preds)
#factors_preds contains the 1/0 layout for all the factors and all the predicted risks values
factors_preds.to_csv('factors_preds.csv') #saving locally

data_tf = identifiers.merge(factors_data_tf, on='personId').merge(prediction_data, on='personId')
#data_tf contains 1 column each for person ID, age, and sex, then the True/False layout for the factors and all the predicted risk values
ipd.display(data_tf)

data = identifiers.merge(factors_data, on='personId').merge(prediction_data, on='personId')
ipd.display(data)
#data contains the same as data_tf, but with the factors data in the original 1/0 format rather than True/False or lists
data.to_csv('data.csv')

,Age group (years)
personId,
1,5-9
2,80-84
3,45-49
4,30-34
5,60-64
...,...
96,50-54
97,5-9
98,90-94


C:\Users\eam565\AppData\Local\Temp\ipykernel_6508\3506572661.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sex_data[i].replace(1, i, inplace=True)
C:\Users\eam565\AppData\Local\Temp\ipykernel_6508\3506572661.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sex_data.replace(sex_data.columns[i], sex[i], inplace=True)


,Factors
personId,
1,Acetaminophen exposures in prior year
1,Hemorrhagic stroke in an inpatient setting in ...
1,ANTIEPILEPTICS in prior year
1,Non-hemorrhagic Stroke in an inpatient setting...
1,Obesity in prior year
...,...
100,Type 1 diabetes and no prior specific non-T1DM...
100,HORMONAL CONTRACEPTIVES in prior year
100,Obesity in prior year


,Factors
personId,
1,"[Acetaminophen exposures in prior year, Hemorr..."
2,"[Seizure in prior year, Sleep apnea in prior y..."
3,"[Coronary artery disease (CAD) in prior year, ..."
4,"[Occurrence of Anxiety in prior year, Antibiot..."
5,"[ANTIEPILEPTICS in prior year, Type 1 diabetes..."
...,...
96,"[Antibiotics Streptogramins in prior year, Low..."
97,"[Hypertension in prior year, Occurrence of Alc..."
98,"[Antibiotics Monobactams in prior year, Osteoa..."


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


,personId,Age group (years),Sex,Factors,predicted risk of Pulmonary Embolism,"predicted risk of Sudden Hearing Loss, No congenital anomaly or middle or inner ear conditions",predicted risk of Restless Leg Syndrome,"predicted risk of Sudden Vision Loss, with no eye pathology causes",predicted risk of Muscle weakness or injury,predicted risk of Ankylosing Spondylitis,predicted risk of Autoimmune hepatitis,predicted risk of Multiple Sclerosis,"predicted risk of Acute pancreatitis, with No chronic or hereditary or common causes of pancreatitis",predicted risk of Ulcerative colitis,predicted risk of Migraine,predicted risk of Dementia,predicted risk of Treatment resistant depression (TRD),"predicted risk of Parkinson's disease, inpatient or with 2nd diagnosis"
0,1,5-9,Female,"[Acetaminophen exposures in prior year, Hemorr...",0.000007,0.001188,0.001135,0.000112,0.018832,0.000076,0.000080,0.000469,0.000124,0.000231,0.006544,0.000073,0.000394,0.000040
1,2,80-84,Male,"[Seizure in prior year, Sleep apnea in prior y...",0.004414,0.035826,0.006286,0.001607,0.179579,0.000502,0.000008,0.000421,0.000817,0.001567,0.024364,0.283879,0.013889,0.019553
2,3,45-49,Male,"[Coronary artery disease (CAD) in prior year, ...",0.002462,0.003524,0.001240,0.000146,0.022318,0.000462,0.000023,0.000527,0.000417,0.001275,0.005430,0.001282,0.001017,0.000113
3,4,30-34,Male,"[Occurrence of Anxiety in prior year, Antibiot...",0.002670,0.002473,0.000441,0.000153,0.022062,0.000482,0.000034,0.000911,0.000236,0.006386,0.006637,0.000707,0.002266,0.000041
4,5,60-64,Other,"[ANTIEPILEPTICS in prior year, Type 1 diabetes...",0.020682,0.011789,0.004277,0.000326,0.063311,0.004115,0.000083,0.001859,0.000700,0.062065,0.004683,0.016372,0.000056,0.003312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,50-54,Female,"[Antibiotics Streptogramins in prior year, Low...",0.002682,0.005175,0.003796,0.000198,0.018149,0.000411,0.000041,0.000423,0.000388,0.001664,0.012047,0.001152,0.001375,0.000154
96,97,5-9,Male,"[Hypertension in prior year, Occurrence of Alc...",0.000007,0.001084,0.000509,0.000045,0.008068,0.000520,0.000011,0.000036,0.000089,0.000385,0.000700,0.000016,0.000191,0.000030
97,98,90-94,Male,"[Antibiotics Monobactams in prior year, Osteoa...",0.012757,0.018546,0.001657,0.000597,0.217159,0.001261,0.000092,0.000219,0.002008,0.002333,0.000461,0.303624,0.000485,0.002572
98,99,85-89,Male,"[Antibiotics Penicillins in prior year, Rheuma...",0.004103,0.025053,0.002515,0.000403,0.056540,0.001541,0.000046,0.000192,0.000317,0.001371,0.002193,0.047227,0.000403,0.001963


,Acetaminophen exposures in prior year,Occurrence of Alcoholism in prior year,Anemia in prior year,Angina events in prior year,ANTIEPILEPTICS in prior year,Occurrence of Anxiety in prior year,Osteoarthritis in prior year,Aspirin exposures in prior year,Occurrence of Asthma in prior year,"Atrial Fibrillation, incident in prior year",...,predicted risk of Muscle weakness or injury,predicted risk of Ankylosing Spondylitis,predicted risk of Autoimmune hepatitis,predicted risk of Multiple Sclerosis,"predicted risk of Acute pancreatitis, with No chronic or hereditary or common causes of pancreatitis",predicted risk of Ulcerative colitis,predicted risk of Migraine,predicted risk of Dementia,predicted risk of Treatment resistant depression (TRD),"predicted risk of Parkinson's disease, inpatient or with 2nd diagnosis"
personId,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,1,1,0,1,0,0,0,...,0.018832,0.000076,0.000080,0.000469,0.000124,0.000231,0.006544,0.000073,0.000394,0.000040
2,0,0,0,0,1,0,0,0,1,1,...,0.179579,0.000502,0.000008,0.000421,0.000817,0.001567,0.024364,0.283879,0.013889,0.019553
3,0,0,0,0,0,0,0,0,0,1,...,0.022318,0.000462,0.000023,0.000527,0.000417,0.001275,0.005430,0.001282,0.001017,0.000113
4,0,0,0,0,0,1,0,1,0,0,...,0.022062,0.000482,0.000034,0.000911,0.000236,0.006386,0.006637,0.000707,0.002266,0.000041
5,0,0,1,0,1,0,0,0,0,0,...,0.063311,0.004115,0.000083,0.001859,0.000700,0.062065,0.004683,0.016372,0.000056,0.003312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,0,0,0,0,0,1,0,...,0.018149,0.000411,0.000041,0.000423,0.000388,0.001664,0.012047,0.001152,0.001375,0.000154
97,0,1,0,0,0,0,0,0,0,0,...,0.008068,0.000520,0.000011,0.000036,0.000089,0.000385,0.000700,0.000016,0.000191,0.000030
98,0,0,0,0,0,0,1,0,0,0,...,0.217159,0.001261,0.000092,0.000219,0.002008,0.002333,0.000461,0.303624,0.000485,0.002572


,personId,Age group (years),Sex,Acetaminophen exposures in prior year,Occurrence of Alcoholism in prior year,Anemia in prior year,Angina events in prior year,ANTIEPILEPTICS in prior year,Occurrence of Anxiety in prior year,Osteoarthritis in prior year,...,predicted risk of Muscle weakness or injury,predicted risk of Ankylosing Spondylitis,predicted risk of Autoimmune hepatitis,predicted risk of Multiple Sclerosis,"predicted risk of Acute pancreatitis, with No chronic or hereditary or common causes of pancreatitis",predicted risk of Ulcerative colitis,predicted risk of Migraine,predicted risk of Dementia,predicted risk of Treatment resistant depression (TRD),"predicted risk of Parkinson's disease, inpatient or with 2nd diagnosis"
0,1,5-9,Female,True,False,False,True,True,False,True,...,0.018832,0.000076,0.000080,0.000469,0.000124,0.000231,0.006544,0.000073,0.000394,0.000040
1,2,80-84,Male,False,False,False,False,True,False,False,...,0.179579,0.000502,0.000008,0.000421,0.000817,0.001567,0.024364,0.283879,0.013889,0.019553
2,3,45-49,Male,False,False,False,False,False,False,False,...,0.022318,0.000462,0.000023,0.000527,0.000417,0.001275,0.005430,0.001282,0.001017,0.000113
3,4,30-34,Male,False,False,False,False,False,True,False,...,0.022062,0.000482,0.000034,0.000911,0.000236,0.006386,0.006637,0.000707,0.002266,0.000041
4,5,60-64,Other,False,False,True,False,True,False,False,...,0.063311,0.004115,0.000083,0.001859,0.000700,0.062065,0.004683,0.016372,0.000056,0.003312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,50-54,Female,False,False,False,False,False,False,False,...,0.018149,0.000411,0.000041,0.000423,0.000388,0.001664,0.012047,0.001152,0.001375,0.000154
96,97,5-9,Male,False,True,False,False,False,False,False,...,0.008068,0.000520,0.000011,0.000036,0.000089,0.000385,0.000700,0.000016,0.000191,0.000030
97,98,90-94,Male,False,False,False,False,False,False,True,...,0.217159,0.001261,0.000092,0.000219,0.002008,0.002333,0.000461,0.303624,0.000485,0.002572
98,99,85-89,Male,False,False,False,False,False,False,True,...,0.056540,0.001541,0.000046,0.000192,0.000317,0.001371,0.002193,0.047227,0.000403,0.001963


,personId,Age group (years),Sex,Acetaminophen exposures in prior year,Occurrence of Alcoholism in prior year,Anemia in prior year,Angina events in prior year,ANTIEPILEPTICS in prior year,Occurrence of Anxiety in prior year,Osteoarthritis in prior year,...,predicted risk of Muscle weakness or injury,predicted risk of Ankylosing Spondylitis,predicted risk of Autoimmune hepatitis,predicted risk of Multiple Sclerosis,"predicted risk of Acute pancreatitis, with No chronic or hereditary or common causes of pancreatitis",predicted risk of Ulcerative colitis,predicted risk of Migraine,predicted risk of Dementia,predicted risk of Treatment resistant depression (TRD),"predicted risk of Parkinson's disease, inpatient or with 2nd diagnosis"
0,1,5-9,Female,1,0,0,1,1,0,1,...,0.018832,0.000076,0.000080,0.000469,0.000124,0.000231,0.006544,0.000073,0.000394,0.000040
1,2,80-84,Male,0,0,0,0,1,0,0,...,0.179579,0.000502,0.000008,0.000421,0.000817,0.001567,0.024364,0.283879,0.013889,0.019553
2,3,45-49,Male,0,0,0,0,0,0,0,...,0.022318,0.000462,0.000023,0.000527,0.000417,0.001275,0.005430,0.001282,0.001017,0.000113
3,4,30-34,Male,0,0,0,0,0,1,0,...,0.022062,0.000482,0.000034,0.000911,0.000236,0.006386,0.006637,0.000707,0.002266,0.000041
4,5,60-64,Other,0,0,1,0,1,0,0,...,0.063311,0.004115,0.000083,0.001859,0.000700,0.062065,0.004683,0.016372,0.000056,0.003312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,50-54,Female,0,0,0,0,0,0,0,...,0.018149,0.000411,0.000041,0.000423,0.000388,0.001664,0.012047,0.001152,0.001375,0.000154
96,97,5-9,Male,0,1,0,0,0,0,0,...,0.008068,0.000520,0.000011,0.000036,0.000089,0.000385,0.000700,0.000016,0.000191,0.000030
97,98,90-94,Male,0,0,0,0,0,0,1,...,0.217159,0.001261,0.000092,0.000219,0.002008,0.002333,0.000461,0.303624,0.000485,0.002572
98,99,85-89,Male,0,0,0,0,0,0,1,...,0.056540,0.001541,0.000046,0.000192,0.000317,0.001371,0.002193,0.047227,0.000403,0.001963
